In [2]:
import sys
import os
# pwd
sys.path

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0',
 'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\TextSummarizerProject\\vir_env',
 '',
 'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\TextSummarizerProject\\vir_env\\Lib\\site-packages',
 'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\TextSummarizerProject\\vir_env\\Lib\\site-packages\\win32',
 'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\TextSummarizerProject\\vir_

In [8]:
os.chdir("../")
# src_path = os.path.abspath(os.path.join(os.getcwd() ,"../"))
# sys.path.append(src_path)
# src_path = os.path.abspath(os.path.join(os.getcwd() ,"../src"))
# sys.path.append(src_path)
# sys.path

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file: Path
    unzip_dir:Path

In [ ]:
# %load_ext autoreload

# %autoreload 2

from textSummarizer.constants.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH, MY_CONSTANT
# from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories, get_file_path
# print(os.getcwd())
# import importlib
# importlib.reload(textSummarizer) 
# importlib.reload(textSummarizer.utils.common)
# MY_CONSTANT


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\TextSummarizerProject\\research'

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):       
        # get_file_path()
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )
        return data_ingestion_config
# test = ConfigurationManager()
    

In [23]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from urllib.error import HTTPError, URLError


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                # print(self.config.source_URL)
                filename,headers = request.urlretrieve(url = self.config.source_URL, filename=self.config.local_data_file)
                logger.info(f"{filename} download! with following info: \n{headers}")
            else:
                logger.info(f"File {Path(self.config.local_data_file)} already exists of size {get_size(Path(self.config.local_data_file))}")
        except HTTPError as e:
            if e.code == 404:
                print(f"HTTP Error 404: The requested resource at {url} was not found.")
            else:
                print(f"HTTP Error {e.code}: {e.reason} for URL: {url}")
        except URLError as e:
            print(f"URL Error: {e.reason} for URL: {url}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted all the files to {unzip_path}")
    def download_extract_zip_file(self):
        zip_path, _ = request.urlretrieve(self.config.source_URL)
        logger.info(f"{zip_path} download! with following info:")
        with zipfile.ZipFile(zip_path, "r") as f:
            f.extractall(self.config.unzip_dir)   

            

In [51]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    # data_ingestion.download_extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e


[2025-06-24 14:06:42,418:INFO:common:yaml file: config\config.yaml loaded successfully.]
[2025-06-24 14:06:42,424:INFO:common:created directory at: artifacts/data_ingestion]
[2025-06-24 14:06:43,639:INFO:3323216149:artifacts\data_ingestion_data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6D82:3F9127:496F16:AB8D7C:685A62C3
Via: 1.1 varnish, 1.1 varnish
Expires: Tue, 24 Jun 2025 08:38:09 GMT
Accept-Ranges: bytes
Date: Tue, 24 Jun 2025 08:36:43 GMT
X-Served-By: cache-bom4731-BOM, cache-bom4727-BOM
X-Cache: MISS, HIT
X-Cache-Hits: 0, 0
X-Timer: S1750754204.912345,VS0,VE1
Vary: Authorization,Accept-Enc